In [1]:
!pip install git+https://github.com/facebookresearch/fastText.git

  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-ca9l40t_
  Running command git clone -q https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-ca9l40t_
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3086315 sha256=9487aefa09634eb84879aff8fffaee5cb30cac2854eecc14410708275c4c04f6
  Stored in directory: /tmp/pip-ephem-wheel-cache-f4k6i2qn/wheels/69/f8/19/7f0ab407c078795bc9f86e1f6381349254f86fd7d229902355
Successfully built fasttext


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#get word embedding libary
import fasttext.util
print('Creating fastText...')
word2vec = fasttext.load_model('/content/drive/MyDrive/fast_text/cc.vi.300.bin')
print('fastText created!')

Creating fastText...
fastText created!


In [4]:
import re
def map_number_and_punct(word):
    # check hem va ngach
    num_of_seperate=0
    dem=0
    for char in word:
      if not char.isnumeric() and char!="/":
        dem+=1        
      if char=="/":
        num_of_seperate+=1
    if dem==0:
      if len(word)>1 and num_of_seperate==1:
        return u'<ngach>'
      if len(word)>1 and num_of_seperate>1:
        return u'<hem>'

    # if re.match(r"^[0-9]{2}0{3}$", word):
    #     return u'<postcode>'
    
    if word.isnumeric():
        word = u'<number>'
    elif word in [u',', u'<', u'.', u'>', u'/', u'?', u'..', u'...', u'....', u':', u';', u'"', u"'", u'[', u'{', u']',
                  u'}', u'|', u'\\', u'`', u'~', u'!', u'@', u'#', u'$', u'%', u'^', u'&', u'*', u'(', u')', u'-', u'+',
                  u'=']:
        word = u'<punct>'

    # word = word.replace("_"," ")
    
    return word

In [5]:
import numpy as np
import codecs
def read_data(input_file):
    with codecs.open(input_file, 'r', 'utf-8') as f:
        word_list = [] 
        words = []
        for line in f:
            line = line.split()
            if len(line) > 0:
                words.append( map_number_and_punct(line[0].lower()) )
            else:
                word_list.append(words)
                words = []
    return word_list

In [6]:
def construct_tensor_word(word_sentences, unknown_embedd, embedd_dim, max_length,ouput_file):
    fileout = open(ouput_file, 'a+', encoding='utf8')
    X = np.empty([1, max_length, embedd_dim])
    for i in range(len(word_sentences)):
        words = word_sentences[i] # words is a sentence
        # print("=====sentence "+str(i)+" ==========")
        length = len(words)
        # run through all word in a sentence
        for j in range(length):
            word = words[j].lower()
            while True:
                try:
                    embedd = word2vec.get_word_vector(query).tolist()
                    embedd = np.ones([1, embedd_dim])
                    break
                except:
                    embedd = unknown_embedd[0]
                    break         
            X[0, j, :] = embedd

        X[0, length:] = np.zeros([1, embedd_dim])
        # print(X.shape)

        np.savetxt(fileout, X[0])
    fileout.close()
    return X

In [7]:
import math
import os.path
from os import path

link_folder = "/content/drive/MyDrive/fast_text/split data/data_0"
index = 0
while path.exists(link_folder):
  print("DATA {}".format(index))
  
  link_train = "/content/drive/MyDrive/fast_text/split data/data_{}/train/data_no_tag.txt".format(index)
  link_val = "/content/drive/MyDrive/fast_text/split data/data_{}/val/data_no_tag.txt".format(index)
  link_test = "/content/drive/MyDrive/fast_text/split data/data_{}/test/data_no_tag_pred.txt".format(index)
  out_train = "/content/drive/MyDrive/fast_text/split data/data_{}/train/word_embedd.txt".format(index)
  out_val = "/content/drive/MyDrive/fast_text/split data/data_{}/val/word_embedd.txt".format(index)
  out_test = "/content/drive/MyDrive/fast_text/split data/data_{}/test/word_embedd.txt".format(index)

  val_word_sentences = read_data(link_val)
  train_word_sentences = read_data(link_train)
  test_word_sentences = read_data(link_test)

  unknown_embedd = np.random.uniform(-0.01, 0.01, [1, 300])
  word_train = construct_tensor_word(train_word_sentences, unknown_embedd, 300, 42,out_train)
  word_val = construct_tensor_word(val_word_sentences, unknown_embedd, 300, 42,out_val)
  word_test = construct_tensor_word(test_word_sentences, unknown_embedd, 300, 42, out_test)

  index+=1
  link_folder = "/content/drive/MyDrive/fast_text/split data/data_{}".format(index)

DATA 0
DATA 1
DATA 2
DATA 3
